In [2]:
import pymysql
from configparser import ConfigParser

# 讀取 .env 檔案取得資料庫連線資訊
config = ConfigParser()
config.read('.env')

# 建立資料庫連線
connection = pymysql.connect(
    host=config.get('DB', 'host'),
    user=config.get('DB', 'user'),
    password=config.get('DB', 'password'),
    port=config.getint('DB', 'port'),
    cursorclass=pymysql.cursors.DictCursor,
)

# 查看資料庫中的所有資料表
with connection.cursor() as cursor:
    cursor.execute("SHOW TABLES FROM superstore;")
    tables = cursor.fetchall()

print(tables)

[{'Tables_in_superstore': 'Customers'}, {'Tables_in_superstore': 'OrderDetails'}, {'Tables_in_superstore': 'Orders'}, {'Tables_in_superstore': 'Products'}]


In [3]:
def sql_query(query):
    """
    執行 SQL 查詢並返回結果
    :param query: SQL 查詢語句
    :return: 查詢結果
    """
    with connection.cursor() as cursor:
        cursor.execute(query)
        result = cursor.fetchall()
        
    return result

In [11]:
category_query = """
SELECT P.category, COUNT(*) as count FROM superstore.Products as P 
JOIN superstore.OrderDetails as O ON P.product_id = O.product_id
GROUP BY category;
"""

category_count_result = sql_query(category_query)
print(category_count_result)

result = [{"label": category["category"], "value": category["count"]} for category in category_count_result]

print(result)

[{'category': 'Furniture', 'count': 2121}, {'category': 'Office Supplies', 'count': 6026}, {'category': 'Technology', 'count': 1847}]
[{'label': 'Furniture', 'value': 2121}, {'label': 'Office Supplies', 'value': 6026}, {'label': 'Technology', 'value': 1847}]


In [12]:
sub_category_query = """
SELECT P.sub_category, COUNT(*) as Count FROM superstore.Products as P
JOIN superstore.OrderDetails as O ON P.product_id = O.product_id
GROUP BY sub_category;
"""

sub_category_count_result = sql_query(sub_category_query)
print(sub_category_count_result)

[{'sub_category': 'Bookcases', 'Count': 228}, {'sub_category': 'Chairs', 'Count': 617}, {'sub_category': 'Furnishings', 'Count': 957}, {'sub_category': 'Tables', 'Count': 319}, {'sub_category': 'Appliances', 'Count': 466}, {'sub_category': 'Art', 'Count': 796}, {'sub_category': 'Binders', 'Count': 1523}, {'sub_category': 'Envelopes', 'Count': 254}, {'sub_category': 'Fasteners', 'Count': 217}, {'sub_category': 'Labels', 'Count': 364}, {'sub_category': 'Paper', 'Count': 1370}, {'sub_category': 'Storage', 'Count': 846}, {'sub_category': 'Supplies', 'Count': 190}, {'sub_category': 'Accessories', 'Count': 775}, {'sub_category': 'Copiers', 'Count': 68}, {'sub_category': 'Machines', 'Count': 115}, {'sub_category': 'Phones', 'Count': 889}]


In [20]:
profit_query = """
SELECT 
    P.category,
    P.sub_category, 
    SUM(O.profit) AS profit
FROM 
    superstore.Products AS P
JOIN 
    superstore.OrderDetails AS O 
    ON P.product_id = O.product_id
GROUP BY 
    P.category, P.sub_category;

"""
profit_count_result = sql_query(profit_query)
print(profit_count_result)

result = {
    "name": "Profit",
    "children": []
}

for profit_count in profit_count_result:
    category = profit_count["category"]
    sub_category = profit_count["sub_category"]
    profit = profit_count["profit"]
    
    # 檢查 category 是否已存在於 result 中
    category_found = False
    for child in result["children"]:
        if child["name"] == category:
            category_found = True
            break
    
    # 如果 category 不存在，則新增一個
    if not category_found:
        result["children"].append({
            "name": category,
            "children": []
        })
    
    # 將 sub_category 和 profit 新增到對應的 category 中
    for child in result["children"]:
        if child["name"] == category:
            child["children"].append({
                "name": sub_category,
                "value": profit
            })
# 印出結果
print(result)


[{'category': 'Furniture', 'sub_category': 'Bookcases', 'profit': Decimal('-3472.56')}, {'category': 'Furniture', 'sub_category': 'Chairs', 'profit': Decimal('26590.15')}, {'category': 'Furniture', 'sub_category': 'Furnishings', 'profit': Decimal('13059.25')}, {'category': 'Furniture', 'sub_category': 'Tables', 'profit': Decimal('-17725.59')}, {'category': 'Office Supplies', 'sub_category': 'Appliances', 'profit': Decimal('18138.07')}, {'category': 'Office Supplies', 'sub_category': 'Art', 'profit': Decimal('6527.96')}, {'category': 'Office Supplies', 'sub_category': 'Binders', 'profit': Decimal('30221.64')}, {'category': 'Office Supplies', 'sub_category': 'Envelopes', 'profit': Decimal('6964.10')}, {'category': 'Office Supplies', 'sub_category': 'Fasteners', 'profit': Decimal('949.53')}, {'category': 'Office Supplies', 'sub_category': 'Labels', 'profit': Decimal('5546.18')}, {'category': 'Office Supplies', 'sub_category': 'Paper', 'profit': Decimal('34053.34')}, {'category': 'Office S

In [9]:
profit_sales_query = """
SELECT category, sub_category, product_name, SUM(sales) AS sales, SUM(profit) AS profit FROM superstore.OrderDetails AS O
JOIN superstore.Products AS P ON O.product_id = P.product_id
GROUP BY category, sub_category, product_name
LIMIT 10;
"""

profit_sales_result = sql_query(profit_sales_query)
print(profit_sales_result)

[{'category': 'Furniture', 'sub_category': 'Bookcases', 'product_name': 'Bush Birmingham Collection Bookcase, Dark Cherry', 'sales': Decimal('825.17'), 'profit': Decimal('-117.88')}, {'category': 'Furniture', 'sub_category': 'Bookcases', 'product_name': 'Sauder Camden County Barrister Bookcase, Planked Cherry Finish', 'sales': Decimal('1064.62'), 'profit': Decimal('24.19')}, {'category': 'Furniture', 'sub_category': 'Bookcases', 'product_name': 'Sauder Inglewood Library Bookcases', 'sales': Decimal('2154.35'), 'profit': Decimal('311.18')}, {'category': 'Furniture', 'sub_category': 'Bookcases', 'product_name': "O'Sullivan 2-Shelf Heavy-Duty Bookcases", 'sales': Decimal('723.85'), 'profit': Decimal('-133.11')}, {'category': 'Furniture', 'sub_category': 'Bookcases', 'product_name': 'Hon Metal Bookcases, Gray', 'sales': Decimal('851.76'), 'profit': Decimal('229.97')}, {'category': 'Furniture', 'sub_category': 'Bookcases', 'product_name': "O'Sullivan Plantations 2-Door Library in Landvery O